In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import polars as pl

In [5]:
# Test Data
X_train = torch.tensor([
    [2, 1, 0],
    [1, 3, 1],
    [2, 2, 2],
    [2, 2, 4],
    [3, 3, 0]
], dtype=torch.float32)

# Corresponding CPU and RAM usage
# 10, 10, 100
# 40, 10, 5
y_train = torch.tensor([
    [30, 100],
    [140, 75],
    [240, 110],
    [440, 120],
    [60, 150]
], dtype=torch.float32)

df = pl.read_csv("training.csv")
X_df = torch.tensor(df.select(["hyperfaas-thumbnailer-json:latest_active_calls","hyperfaas-echo:latest_active_calls","hyperfaas-bfs-json:latest_active_calls"]).to_numpy(), dtype=torch.float32)
Y_df =  torch.tensor(df.select(["cpu_usage","memory_usage"]).to_numpy(), dtype=torch.float32)

In [3]:
class ResourceUsageModel(nn.Module):
    def __init__(self, input_size):
        super(ResourceUsageModel, self).__init__()
        # Single linear layer: input_size -> 2 outputs
        self.linear = nn.Linear(input_size, 2)

    def forward(self, x):
        return self.linear(x)

In [7]:
input_size = 3
model = ResourceUsageModel(input_size)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_df)
    loss = criterion(outputs, Y_df)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 200 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Example prediction
with torch.no_grad():
    test_input = torch.tensor([[0, 1, 0]], dtype=torch.float32)
    predicted = model(test_input)
    print("Predicted CPU and RAM usage:", predicted.numpy())

Epoch [200/1000], Loss: 831.6086
Epoch [400/1000], Loss: 777.7339
Epoch [600/1000], Loss: 775.9797
Epoch [800/1000], Loss: 775.6640
Epoch [1000/1000], Loss: 775.5357
Predicted CPU and RAM usage: [[ 0.514303 87.45232 ]]
